In [1]:
import numpy as np
import pandas as pd
import sklearn
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import tqdm
from tqdm import tqdm

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from facenet_pytorch import MTCNN, InceptionResnetV1

num_classes = 31

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
resnet = InceptionResnetV1(pretrained='vggface2', classify = True, num_classes=num_classes)
resnet

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

In [3]:
model_name = "resnet_model_data_augmentation.pth"
data_dir = '/home/bishal/demoprojects/facerecognition/Dataset/Faces'
video_file = "/home/bishal/demoprojects/facerecognition/Dataset/Lisa_and_Ellen_FHD1.mp4"
save_video_file = "Det_n_Rec_Lisa_and_Ellen_FHD.mp4"

resnet.load_state_dict(torch.load(model_name))

# Set the model to evaluation mode
resnet.eval()




InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

In [4]:
from facenet_pytorch import MTCNN
import torch
import numpy as np
import mmcv, cv2
from PIL import Image, ImageDraw
from IPython import display

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [6]:
mtcnn = MTCNN(keep_all=True, device=device)

In [7]:
from torchvision.datasets import ImageFolder


dataset = ImageFolder(root=data_dir, transform=None)
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}




In [8]:

video = mmcv.VideoReader(video_file)

# display.Video(video_file, width=640, embed=True)

In [9]:
from PIL import Image, ImageDraw, ImageFont
import torch 


detection_threshold = 0.9
recognition_threshold = 0.9

softmax = torch.nn.Softmax(dim=1).eval()
frames_tracked = []

resnet.to(device)


font = ImageFont.load_default(size=30.0)

for i, frame in enumerate(video):

    if i < 5500:
        frame = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        print('\rTracking frame: {}'.format(i + 1), end='')
        
        # Detect faces
        # boxes, _ = mtcnn.detect(frame)

        # Detect faces
        boxes, batch_probs, batch_points = mtcnn.detect(frame, landmarks=True)

        # boxes = boxes.tolist()
        
        # filtering detected based on their probability

        # if boxes is not None:
        #     for i, det_prob in enumerate(batch_probs):
        #         if det_prob < detection_threshold:
        #             boxes.pop(i)
        #     if len(boxes) == 0:
        #         boxes = None
        #     else:
        #         boxes = np.array(boxes)

        # Extract faces
        faces = mtcnn.extract(frame, boxes, save_path=None) # of shape Bx3x160x160
        

        
        # Select faces
        # boxes, batch_probs, batch_points = mtcnn.select_boxes(
        #     boxes, batch_probs, batch_points, frame, method="probability", threshold=0.9)
        
        # print(batch_boxes.shape)

        


        # Draw faces and labels
        frame_draw = frame.copy()
        draw = ImageDraw.Draw(frame_draw)



        if boxes is not None:

            # resize Bx3x160x160 to Bx3x224x224

            # Define the transformation to convert a tensor to a PIL image
            # to_pil_image = transforms.ToPILImage()

            # # Loop over each image in the batch and convert to PIL image
            # pil_images = [to_pil_image(faces[i]) for i in range(faces.size(0))]

            # new_faces_list = []
            # for pil_image in pil_images:
            #     new_faces_list.append(transform(pil_image))

            # faces = torch.stack(new_faces_list)

            # perform recognition over detected faces
            faces = faces.to(device)
            outputs = resnet(faces)
            prob = softmax(outputs)

            predicted_prob, predicted = torch.max(prob.detach(), 1)
            predicted_prob = predicted_prob.cpu().numpy()
            predicted = predicted.cpu().numpy()

            print(predicted_prob)

            # Assuming `predicted` is a list of class indices and `boxes` is a list of bounding boxes
            labels = [dataset.idx_to_class[pred] for pred in predicted]

            
            for i, (box, label) in enumerate(zip(boxes, labels)):
                if predicted_prob[i] > recognition_threshold and batch_probs[i] > detection_threshold:
                    draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6)
                    
                    # Add label
                    text_size = draw.textbbox((0, 0), str(int(predicted_prob[i]*100))+label, font=font)
                    text_location = [box[0], box[1] - text_size[1]]
                    textbox_location = [box[0], box[1] - text_size[1], box[0] + text_size[0], box[1]]
                    
                    draw.rectangle(textbox_location, fill=(255, 0, 0))
                    draw.text(text_location, str(int(predicted_prob[i]*100))+label, fill=(255, 255, 255), font=font)

        # Add to frame list
        # frames_tracked.append(frame_draw.resize((640, 360), Image.BILINEAR))
        frames_tracked.append(frame_draw)

print('\nDone')

dim = frames_tracked[0].size
fourcc = cv2.VideoWriter_fourcc(*'FMP4')    
video_tracked = cv2.VideoWriter(save_video_file, fourcc, 25.0, dim)
for frame in frames_tracked:
    video_tracked.write(cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR))
video_tracked.release()


Tracking frame: 1[0.6837143 0.9997414]
Tracking frame: 2[0.9893155 0.9978757]
Tracking frame: 3[0.99909484 0.9610332 ]
Tracking frame: 4[0.99690646]
Tracking frame: 5[0.9602605]
Tracking frame: 6[0.99862015]
Tracking frame: 7[0.99839705]
Tracking frame: 8[0.9977276]
Tracking frame: 9[0.99955374]
Tracking frame: 10[0.9994686]
Tracking frame: 11[0.99997675]
Tracking frame: 12[0.99988604 0.46772876]
Tracking frame: 13[0.9999157]
Tracking frame: 14[0.99989474]
Tracking frame: 15[0.9999062]
Tracking frame: 16[0.99986935]
Tracking frame: 17[0.99983656 0.25432616]
Tracking frame: 18[0.9996289 0.9996748]
Tracking frame: 19[0.99952483]
Tracking frame: 20[0.99992216]
Tracking frame: 21[0.9999889]
Tracking frame: 22[0.99992716 0.6744816 ]
Tracking frame: 23[0.9999862]
Tracking frame: 24[0.9999876]
Tracking frame: 25[0.9999697  0.27610123]
Tracking frame: 26[0.9999764]
Tracking frame: 27[0.838068   0.99991095]
Tracking frame: 28[0.99986756]
Tracking frame: 29[0.99964535]
Tracking frame: 30[0.99941